In [1]:
!nvidia-smi

Thu Jan 23 18:24:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.26       Driver Version: 430.26       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1D:00.0 Off |                  N/A |
| 27%   26C    P8    13W / 250W |     10MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1E:00.0 Off |                  N/A |
| 27%   25C    P8     6W / 250W |     10MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('/mnt/disk1/project/SMhospital/capsule/ce_packages')

from ce_utils import load_data, one_hot, test_batch_idxs, batch_flatten
from ce_utils import model_prob
from ce_utils import classification_metric
from ce_model import binary_model, ensemble_model

from scipy.stats import sem, t
from scipy import mean
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

# Load Data

In [3]:
import glob
import cv2

def load_data(phase, cls, les = None, data = 'sm', data_dir = '/mnt/disk2/data/private_data/SMhospital/capsule/1 preprocessed', 
              extract_name = True, image_ch = 'rgb'):
    """
    phase = 'train', 'test'
    cls: [les]  
      'n': ['neg']
      'h': ['redspot', 'angio', 'active'], 
      'd': ['ero', 'ulc', 'str'],
      'p': ['amp', 'lym', 'tum']}
    """
    lesions = dict(neg = 'negative', 
                   redspot = 'red_spot', angio = 'angioectasia', active = 'active_bleeding', 
                   ero = 'erosion', ulcer = 'ulcer', str = 'stricture', 
                   amp = 'ampulla_of_vater', lym = 'lymphoid_follicles', tum = 'small_bowel_tumor')
    classes = dict(n = 'negative', h = 'hemorrhagic', d = 'depressed', p = 'protruded')

    path = os.path.join(data_dir, data, phase, classes[cls], lesions[les])
    pathlist = glob.glob(path + '/*.jpg')
    
    new_pathlist = []
    for p in pathlist:
        name = os.path.basename(p)
        name_seg1 = name.split('___')[0]
        if not ('f_' in name_seg1 or 'r1_' in name_seg1 or 'r2_' in name_seg1 or 'r3_' in name_seg1):
            new_pathlist.append(p)
    return load_image_from_path(new_pathlist, image_ch = image_ch, extract_name = extract_name)

def load_image_from_path(pathlist,image_ch = 'bgr', extract_name = False):
    data = []
    for i in pathlist:
        temp = cv2.imread(i)
        if image_ch == 'bgr':
            pass
        elif image_ch == 'rgb':
            temp = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
        elif image_ch == 'hsv':
            temp = cv2.cvtColor(temp, cv2.COLOR_BGR2HSV)
        data.append(temp)
    if extract_name != False:
        name = []
        for i in pathlist:
            name.append(os.path.basename(i))
        return np.asarray(data), np.asarray(name)
    else:
        return np.asarray(data) 

In [4]:
data_dir = '/mnt/disk2/data/private_data/SMhospital/capsule/0 data/labeled'

neg_rgb_img, neg_name = load_data('preprocessing', 'n', 'neg', data = '200121 validation dataset', data_dir = data_dir, image_ch = 'rgb')
redspot_rgb_img, redspot_name = load_data('preprocessing', 'h', 'redspot', data = '200121 validation dataset', data_dir = data_dir, image_ch = 'rgb')
angio_rgb_img, angio_name = load_data('preprocessing', 'h', 'angio', data = '200121 validation dataset', data_dir = data_dir, image_ch = 'rgb')
active_rgb_img, active_name = load_data('preprocessing', 'h', 'active', data = '200121 validation dataset', data_dir = data_dir, image_ch = 'rgb')
ero_rgb_img, ero_name = load_data('preprocessing', 'd', 'ero', data = '200121 validation dataset', data_dir = data_dir, image_ch = 'rgb')
ulcer_rgb_img, ulcer_name = load_data('preprocessing', 'd', 'ulcer', data = '200121 validation dataset', data_dir = data_dir, image_ch = 'rgb')
str_rgb_img, str_name = load_data('preprocessing', 'd', 'str', data = '200121 validation dataset', data_dir = data_dir, image_ch = 'rgb')

neg_bgr_img, _ = load_data('preprocessing', 'n', 'neg', data = '200121 validation dataset', data_dir = data_dir, image_ch = 'bgr')
redspot_bgr_img, _ = load_data('preprocessing', 'h', 'redspot', data = '200121 validation dataset', data_dir = data_dir, image_ch = 'bgr')
angio_bgr_img, _ = load_data('preprocessing', 'h', 'angio', data = '200121 validation dataset', data_dir = data_dir, image_ch = 'bgr')
active_bgr_img, _ = load_data('preprocessing', 'h', 'active', data = '200121 validation dataset', data_dir = data_dir, image_ch = 'bgr')
ero_bgr_img, _ = load_data('preprocessing', 'd', 'ero', data = '200121 validation dataset', data_dir = data_dir, image_ch = 'bgr')
ulcer_bgr_img, _ = load_data('preprocessing', 'd', 'ulcer', data = '200121 validation dataset', data_dir = data_dir, image_ch = 'bgr')
str_bgr_img, _ = load_data('preprocessing', 'd', 'str', data = '200121 validation dataset', data_dir = data_dir, image_ch = 'bgr')

In [5]:
neg_label = one_hot(neg_rgb_img, 0)
redspot_label = one_hot(redspot_rgb_img, 1)
angio_label = one_hot(angio_rgb_img, 1)
active_label = one_hot(active_rgb_img, 1)
ero_label = one_hot(ero_rgb_img, 1)
ulcer_label = one_hot(ulcer_rgb_img, 1)
str_label = one_hot(str_rgb_img, 1)

print("red spot:",neg_rgb_img.shape, neg_label.shape)
print("red spot:",redspot_rgb_img.shape, redspot_label.shape)
print("angioectasia:",angio_rgb_img.shape, angio_label.shape)
print("active_bleeding:",active_rgb_img.shape, active_label.shape)
print("erosion:",ero_rgb_img.shape, ero_label.shape)
print("ulcer:",ulcer_rgb_img.shape, ulcer_label.shape)
print("stricture:",str_rgb_img.shape, str_label.shape)

red spot: (1005, 512, 512, 3) (1005, 2)
red spot: (370, 512, 512, 3) (370, 2)
angioectasia: (2, 512, 512, 3) (2, 2)
active_bleeding: (293, 512, 512, 3) (293, 2)
erosion: (297, 512, 512, 3) (297, 2)
ulcer: (410, 512, 512, 3) (410, 2)
stricture: (48, 512, 512, 3) (48, 2)


In [6]:
h_label = np.vstack([redspot_label, angio_label, active_label])
d_label = np.vstack([ero_label, ulcer_label, str_label])
labels = np.vstack([neg_label, h_label, d_label])

# Import Model

In [7]:
nh_core_list = sorted([i[:-5] for i in os.listdir('./model/CORE') if 'meta' in i and 'CORE_rgb_NH' in i])
nd_core_list = sorted([i[:-5] for i in os.listdir('./model/CORE') if 'meta' in i and 'CORE_rgb_ND' in i])
b_core_list = sorted([i[:-5] for i in os.listdir('./model/CORE') if 'meta' in i and 'CORE_rgb_B' in i])

In [8]:
nh_list = sorted([i[:-5] for i in os.listdir('./model/ENTIRE') if 'meta' in i and 'ENTIRE_rgb_NH' in i])
nd_list = sorted([i[:-5] for i in os.listdir('./model/ENTIRE') if 'meta' in i and 'ENTIRE_rgb_ND' in i])
b_list = sorted([i[:-5] for i in os.listdir('./model/ENTIRE') if 'meta' in i and 'ENTIRE_rgb_B' in i])

In [10]:
nh_2_list = sorted([i[:-5] for i in os.listdir('./model/ENTIRE2') if 'meta' in i and 'ENTIRE2_NH' in i])
nd_2_list = sorted([i[:-5] for i in os.listdir('./model/ENTIRE2') if 'meta' in i and 'ENTIRE2_ND' in i])
b_2_list = sorted([i[:-5] for i in os.listdir('./model/ENTIRE2') if 'meta' in i and 'ENTIRE2_B' in i])

In [11]:
nh_t_list = sorted([i[:-5] for i in os.listdir('./model/ENTIRE(TOTAL)') if 'meta' in i and 'ENTIRE(TOTAL)_NH' in i])
nd_t_list = sorted([i[:-5] for i in os.listdir('./model/ENTIRE(TOTAL)') if 'meta' in i and 'ENTIRE(TOTAL)_ND' in i])
b_t_list = sorted([i[:-5] for i in os.listdir('./model/ENTIRE(TOTAL)') if 'meta' in i and 'ENTIRE(TOTAL)_B' in i])

In [12]:
nh_path = './model/CORE/' + nh_core_list[2]
nd_path = './model/CORE/' + nd_core_list[0]
b_path = './model/CORE/' + b_core_list[2]

NH_core_model = binary_model(nh_path)
ND_core_model = binary_model(nd_path)
E_core_model = ensemble_model(nh_path, nd_path)
B_core_model = binary_model(b_path)

nh_path = './model/ENTIRE/' + nh_list[-1]
nd_path = './model/ENTIRE/' + nd_list[-1]
b_path = './model/ENTIRE/' + b_list[-1]

NH_model = binary_model(nh_path)
ND_model = binary_model(nd_path)
E_model = ensemble_model(nh_path, nd_path)
B_model = binary_model(b_path)

nh_path = './model/ENTIRE2/' + nh_2_list[-1]
nd_path = './model/ENTIRE2/' + nd_2_list[-1]
b_path = './model/ENTIRE2/' + b_2_list[-1]

NH_2_model = binary_model(nh_path)
ND_2_model = binary_model(nd_path)
E_2_model = ensemble_model(nh_path, nd_path)
B_2_model = binary_model(b_path)

nh_path = './model/ENTIRE(TOTAL)/' + nh_t_list[4]
nd_path = './model/ENTIRE(TOTAL)/' + nd_t_list[4]
b_path = './model/ENTIRE(TOTAL)/' + b_t_list[4]

NH_t_model = binary_model(nh_path)
ND_t_model = binary_model(nd_path)
E_t_model = ensemble_model(nh_path, nd_path)
B_t_model = binary_model(b_path)

W0123 18:25:57.100864 139873660139264 deprecation_wrapper.py:119] From /mnt/disk1/project/SMhospital/capsule/ce_packages/ce_model.py:108: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0123 18:25:57.416920 139873660139264 deprecation_wrapper.py:119] From /mnt/disk1/project/SMhospital/capsule/ce_packages/ce_model.py:109: The name tf.train.import_meta_graph is deprecated. Please use tf.compat.v1.train.import_meta_graph instead.

W0123 18:25:57.769218 139873660139264 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


# Evaluation

In [13]:
def conf_interval(data, conf_level = 0.95, mode = 'value', metric = 'Accr'):
    n = len(data)
    m = mean(data)
    std_err = sem(data)
    h = std_err * t.ppf((1 + conf_level) / 2, n - 1)

    start = m - h
    end = m + h
    
    if mode == 'value':
        return m, start, end
    elif mode == 'print':
        return print("{} --- {:.2f} ({:.2f} ~ {:.2f})".format(metric, m, start, end))

def k_fold_idx(n_fold, dataset):
    kf = KFold(n_splits = n_fold, shuffle = True)
    train_idxs = []
    test_idxs = []
    for train_idx, test_idx in kf.split(dataset):
        train_idxs.append(train_idx)
        test_idxs.append(test_idx)
    return train_idxs, test_idxs

def tri_shuffle(x, y, z):
    rand_idx = np.random.choice(len(x), len(x), replace = False)
    return x[rand_idx], y[rand_idx], z[rand_idx]

def extract_5metrics(labels, outputs):
    true = np.argmax(labels, axis = 1)
    pred = np.argmax(outputs, axis = 1)
    
    accr = np.mean(np.equal(true, pred))
    
    tp = np.intersect1d(np.where(true == 1)[0], np.where(pred == 1)[0])
    tn = np.intersect1d(np.where(true == 0)[0], np.where(pred == 0)[0])
    sens, spec, npv, ppv = 0, 0, 0, 0
    if not len(np.where(true == 1)[0]) == 0:
        sens = len(tp)/len(np.where(true == 1)[0])
    if not len(np.where(true == 0)[0]) == 0:
        spec = len(tn)/len(np.where(true == 0)[0])
    if not len(np.where(pred == 0)[0]) == 0:
        npv = len(tn)/len(np.where(pred == 0)[0])
    if not len(np.where(pred == 1)[0]) == 0:
        ppv = len(tp)/len(np.where(pred == 1)[0])
    
    return [accr, sens, spec, npv, ppv]

In [14]:
print('CORE')
print('------------------------------------------')
print('Model Inference')
print('')
print('Binary model')

neg_b_output = model_prob(B_core_model, neg_rgb_img)
redspot_b_output = model_prob(B_core_model, redspot_rgb_img)
angio_b_output = model_prob(B_core_model, angio_rgb_img)
active_b_output = model_prob(B_core_model, active_rgb_img)
ero_b_output = model_prob(B_core_model, ero_rgb_img)
ulcer_b_output = model_prob(B_core_model, ulcer_rgb_img)
str_b_output  = model_prob(B_core_model, str_rgb_img)

print('')
print('Ensemble model')

neg_e_output, neg_nh_output, neg_nd_output = model_prob(E_core_model, neg_rgb_img, model_type = 'ensemble')
redspot_e_output, redspot_nh_output, redspot_nd_output = model_prob(E_core_model, redspot_rgb_img, model_type = 'ensemble')
angio_e_output, angio_nh_output, angio_nd_output = model_prob(E_core_model, angio_rgb_img, model_type = 'ensemble')
active_e_output, active_nh_output, active_nd_output = model_prob(E_core_model, active_rgb_img, model_type = 'ensemble')
ero_e_output, ero_nh_output, ero_nd_output = model_prob(E_core_model, ero_rgb_img, model_type = 'ensemble')
ulcer_e_output, ulcer_nh_output, ulcer_nd_output = model_prob(E_core_model, ulcer_rgb_img, model_type = 'ensemble')
str_e_output, str_nh_output, str_nd_output  = model_prob(E_core_model, str_rgb_img, model_type = 'ensemble')

h_b_output = np.vstack([redspot_b_output, angio_b_output, active_b_output])
d_b_output = np.vstack([ero_b_output, ulcer_b_output, str_b_output])
b_outputs = np.vstack([neg_b_output, h_b_output, d_b_output])

h_e_output = np.vstack([redspot_e_output, angio_e_output, active_e_output])
d_e_output = np.vstack([ero_e_output, ulcer_e_output, str_e_output])
e_outputs = np.vstack([neg_e_output, h_e_output, d_e_output])

cls = classification_metric()
print('----------------------------------------')
print('Accucay per lesion')
print('')
print('NH model')
neg_nh_acc = cls.accuracy(neg_label, neg_nh_output, 'negative')
redspot_nh_acc = cls.accuracy(redspot_label, redspot_nh_output, 'red spot')
angio_nh_acc = cls.accuracy(angio_label, angio_nh_output, 'angioectasia')
active_nh_acc = cls.accuracy(active_label, active_nh_output, 'active bleeding')
ero_nh_acc = cls.accuracy(ero_label, ero_nh_output, 'erosion')
ulcer_nh_acc = cls.accuracy(ulcer_label, ulcer_nh_output, 'ulcer')
str_nh_acc = cls.accuracy(str_label, str_nh_output, 'stricture')

print('')
print('ND model')
neg_nd_acc = cls.accuracy(neg_label, neg_nd_output, 'negative')
redspot_nd_acc = cls.accuracy(redspot_label, redspot_nd_output, 'red spot')
angio_nd_acc = cls.accuracy(angio_label, angio_nd_output, 'angioectasia')
active_nd_acc = cls.accuracy(active_label, active_nd_output, 'active bleeding')
ero_nd_acc = cls.accuracy(ero_label, ero_nd_output, 'erosion')
ulcer_nd_acc = cls.accuracy(ulcer_label, ulcer_nd_output, 'ulcer')
str_nd_acc = cls.accuracy(str_label, str_nd_output, 'stricture')

print('')
print('Binary model')
neg_b_acc = cls.accuracy(neg_label, neg_b_output, 'negative')
redspot_b_acc = cls.accuracy(redspot_label, redspot_b_output, 'red spot')
angio_b_acc = cls.accuracy(angio_label, angio_b_output, 'angioectasia')
active_b_acc = cls.accuracy(active_label, active_b_output, 'active bleeding')
ero_b_acc = cls.accuracy(ero_label, ero_b_output, 'erosion')
ulcer_b_acc = cls.accuracy(ulcer_label, ulcer_b_output, 'ulcer')
str_b_acc = cls.accuracy(str_label, str_b_output, 'stricture')

print('')
print('Ensemble model')
neg_e_acc = cls.accuracy(neg_label, neg_e_output, 'negative')
redspot_e_acc = cls.accuracy(redspot_label, redspot_e_output, 'red spot')
angio_e_acc = cls.accuracy(angio_label, angio_e_output, 'angioectasia')
active_e_acc = cls.accuracy(active_label, active_e_output, 'active bleeding')
ero_e_acc = cls.accuracy(ero_label, ero_e_output, 'erosion')
ulcer_e_acc = cls.accuracy(ulcer_label, ulcer_e_output, 'ulcer')
str_e_acc = cls.accuracy(str_label, str_e_output, 'stricture')

n_g = 10

_, n_idxs = k_fold_idx(n_fold = n_g, dataset = neg_label)
_, h_idxs = k_fold_idx(n_fold = n_g, dataset = h_label)
_, d_idxs = k_fold_idx(n_fold = n_g, dataset = d_label)


h_label, h_b_output, h_e_output = tri_shuffle(h_label, h_b_output, h_e_output)
d_label, d_b_output, d_e_output = tri_shuffle(d_label, d_b_output, d_e_output)

# e_aucs, b_aucs = [], []
# e_CM, b_CM = [], []

b_metrics = []
e_metrics = []

for n_idx, h_idx, d_idx in zip(n_idxs, h_idxs, d_idxs):

    labels = np.vstack([neg_label[n_idx], h_label[h_idx], d_label[d_idx]])
    b_outputs = np.vstack([neg_b_output[n_idx], h_b_output[h_idx], d_b_output[d_idx]]) 
    e_outputs = np.vstack([neg_e_output[n_idx], h_e_output[h_idx], d_e_output[d_idx]]) 
    
    b_metrics.append(extract_5metrics(labels, b_outputs)), e_metrics.append(extract_5metrics(labels, e_outputs))
    
#     e_fpr, e_tpr, thresholds = roc_curve(labels[:, 1], e_outputs[:,1])
#     b_fpr, b_tpr, thresholds = roc_curve(labels[:, 1], b_outputs[:,1])
#     e_aucs.append(auc(e_fpr, e_tpr)), b_aucs.append(auc(b_fpr, b_tpr))

# b_metrics = []
# e_metrics = []

# for e_cm, b_cm in zip(e_CM, b_CM):
#     b_metrics.append(cls.cm2metric(b_cm))
#     e_metrics.append(cls.cm2metric(e_cm))
    
b_metrics = np.asarray(b_metrics)
e_metrics = np.asarray(e_metrics)
print('----------------------------------------')
print('Binary model VS Ensemble model')
    
print("95 % CI\n")

print('Binary model')
conf_interval(100*b_metrics[:, 0], mode = 'print', metric = 'Accr')
conf_interval(100*b_metrics[:, 1], mode = 'print', metric = 'Sen')
conf_interval(100*b_metrics[:, 2], mode = 'print', metric = 'Spec')
conf_interval(100*b_metrics[:, 3], mode = 'print', metric = 'NPV')
conf_interval(100*b_metrics[:, 4], mode = 'print', metric = 'PPV')
print('')
print('Ensemble model')
conf_interval(100*e_metrics[:, 0], mode = 'print', metric = 'Accr')
conf_interval(100*e_metrics[:, 1], mode = 'print', metric = 'Sen')
conf_interval(100*e_metrics[:, 2], mode = 'print', metric = 'Spec')
conf_interval(100*e_metrics[:, 3], mode = 'print', metric = 'NPV')
conf_interval(100*e_metrics[:, 4], mode = 'print', metric = 'PPV')    
    
# e_auc_m, e_auc_l, e_auc_u = conf_interval(e_aucs, mode = 'value')
# b_auc_m, b_auc_l, b_auc_u = conf_interval(b_aucs, mode = 'value')

# labels, b_outputs, e_outputs = tri_shuffle(labels, b_outputs, e_outputs)

# e_fpr, e_tpr, thresholds = roc_curve(labels[:, 1], e_outputs[:,1])
# b_fpr, b_tpr, thresholds = roc_curve(labels[:, 1], b_outputs[:,1])

# fig = plt.figure(figsize=(10, 8))
# plt.plot(b_fpr, b_tpr, 'k--', label="Binary model, AUC: {:.4f} ({:.4f}~{:.4f}), 95 % CI".format(b_auc_m, b_auc_l, b_auc_u))
# plt.plot(e_fpr, e_tpr, 'r--', label="Ensemble model, AUC: {:.4f} ({:.4f}~{:.4f}), 95 % CI".format(e_auc_m, e_auc_l, round(e_auc_u)))
# plt.xlabel('1 - Specificity', fontsize = 15)
# plt.ylabel('Sensitivity', fontsize = 15)
# plt.legend(fontsize = 12)
# plt.grid('on')
# fig.show()
# # fig.savefig('./results/roc_curve.png', bbox_inches='tight', dpi = 300)

# fig = plt.figure(figsize=(10, 8))
# plt.plot(b_fpr, b_tpr, 'k--', label="Binary model")
# plt.plot(e_fpr, e_tpr, 'r--', label="Ensemble model")
# plt.xlim([0.0, 0.1])
# plt.ylim([0.9, 1.0])
# plt.legend(fontsize = 20, loc = 4)
# plt.axis('off')
# fig.show()
# # fig.savefig('./results/roc_curve_enlarged.png', bbox_inches='tight', dpi = 300)

CORE
------------------------------------------
Model Inference

Binary model
#: 1005 / Inference Time: 00:00:09
#: 370 / Inference Time: 00:00:03
#: 2 / Inference Time: 00:00:00
#: 293 / Inference Time: 00:00:02
#: 297 / Inference Time: 00:00:02
#: 410 / Inference Time: 00:00:03
#: 48 / Inference Time: 00:00:00

Ensemble model
#: 1005 / Inference Time: 00:00:11
#: 370 / Inference Time: 00:00:04
#: 2 / Inference Time: 00:00:00
#: 293 / Inference Time: 00:00:03
#: 297 / Inference Time: 00:00:03
#: 410 / Inference Time: 00:00:04
#: 48 / Inference Time: 00:00:00
----------------------------------------
Accucay per lesion

NH model
negative: 95.52 %
red spot: 60.27 %
angioectasia: 100.00 %
active bleeding: 92.83 %
erosion: 51.85 %
ulcer: 50.00 %
stricture: 39.58 %

ND model
negative: 84.18 %
red spot: 56.76 %
angioectasia: 50.00 %
active bleeding: 77.13 %
erosion: 76.43 %
ulcer: 87.80 %
stricture: 83.33 %

Binary model
negative: 87.16 %
red spot: 72.97 %
angioectasia: 100.00 %
active bleed

In [16]:
print('ENTIRE')
print('------------------------------------------')
print('Model Inference')
print('')
print('Binary model')

neg_b_output = model_prob(B_model, neg_rgb_img)
redspot_b_output = model_prob(B_model, redspot_rgb_img)
angio_b_output = model_prob(B_model, angio_rgb_img)
active_b_output = model_prob(B_model, active_rgb_img)
ero_b_output = model_prob(B_model, ero_rgb_img)
ulcer_b_output = model_prob(B_model, ulcer_rgb_img)
str_b_output  = model_prob(B_model, str_rgb_img)

print('')
print('Ensemble model')

neg_e_output, neg_nh_output, neg_nd_output = model_prob(E_model, neg_rgb_img, model_type = 'ensemble')
redspot_e_output, redspot_nh_output, redspot_nd_output = model_prob(E_model, redspot_rgb_img, model_type = 'ensemble')
angio_e_output, angio_nh_output, angio_nd_output = model_prob(E_model, angio_rgb_img, model_type = 'ensemble')
active_e_output, active_nh_output, active_nd_output = model_prob(E_model, active_rgb_img, model_type = 'ensemble')
ero_e_output, ero_nh_output, ero_nd_output = model_prob(E_model, ero_rgb_img, model_type = 'ensemble')
ulcer_e_output, ulcer_nh_output, ulcer_nd_output = model_prob(E_model, ulcer_rgb_img, model_type = 'ensemble')
str_e_output, str_nh_output, str_nd_output  = model_prob(E_model, str_rgb_img, model_type = 'ensemble')

h_b_output = np.vstack([redspot_b_output, angio_b_output, active_b_output])
d_b_output = np.vstack([ero_b_output, ulcer_b_output, str_b_output])
b_outputs = np.vstack([neg_b_output, h_b_output, d_b_output])

h_e_output = np.vstack([redspot_e_output, angio_e_output, active_e_output])
d_e_output = np.vstack([ero_e_output, ulcer_e_output, str_e_output])
e_outputs = np.vstack([neg_e_output, h_e_output, d_e_output])

cls = classification_metric()
print('----------------------------------------')
print('Accucay per lesion')
print('')
print('NH model')
neg_nh_acc = cls.accuracy(neg_label, neg_nh_output, 'negative')
redspot_nh_acc = cls.accuracy(redspot_label, redspot_nh_output, 'red spot')
angio_nh_acc = cls.accuracy(angio_label, angio_nh_output, 'angioectasia')
active_nh_acc = cls.accuracy(active_label, active_nh_output, 'active bleeding')
ero_nh_acc = cls.accuracy(ero_label, ero_nh_output, 'erosion')
ulcer_nh_acc = cls.accuracy(ulcer_label, ulcer_nh_output, 'ulcer')
str_nh_acc = cls.accuracy(str_label, str_nh_output, 'stricture')

print('')
print('ND model')
neg_nd_acc = cls.accuracy(neg_label, neg_nd_output, 'negative')
redspot_nd_acc = cls.accuracy(redspot_label, redspot_nd_output, 'red spot')
angio_nd_acc = cls.accuracy(angio_label, angio_nd_output, 'angioectasia')
active_nd_acc = cls.accuracy(active_label, active_nd_output, 'active bleeding')
ero_nd_acc = cls.accuracy(ero_label, ero_nd_output, 'erosion')
ulcer_nd_acc = cls.accuracy(ulcer_label, ulcer_nd_output, 'ulcer')
str_nd_acc = cls.accuracy(str_label, str_nd_output, 'stricture')

print('')
print('Binary model')
neg_b_acc = cls.accuracy(neg_label, neg_b_output, 'negative')
redspot_b_acc = cls.accuracy(redspot_label, redspot_b_output, 'red spot')
angio_b_acc = cls.accuracy(angio_label, angio_b_output, 'angioectasia')
active_b_acc = cls.accuracy(active_label, active_b_output, 'active bleeding')
ero_b_acc = cls.accuracy(ero_label, ero_b_output, 'erosion')
ulcer_b_acc = cls.accuracy(ulcer_label, ulcer_b_output, 'ulcer')
str_b_acc = cls.accuracy(str_label, str_b_output, 'stricture')

print('')
print('Ensemble model')
neg_e_acc = cls.accuracy(neg_label, neg_e_output, 'negative')
redspot_e_acc = cls.accuracy(redspot_label, redspot_e_output, 'red spot')
angio_e_acc = cls.accuracy(angio_label, angio_e_output, 'angioectasia')
active_e_acc = cls.accuracy(active_label, active_e_output, 'active bleeding')
ero_e_acc = cls.accuracy(ero_label, ero_e_output, 'erosion')
ulcer_e_acc = cls.accuracy(ulcer_label, ulcer_e_output, 'ulcer')
str_e_acc = cls.accuracy(str_label, str_e_output, 'stricture')

n_g = 10

_, n_idxs = k_fold_idx(n_fold = n_g, dataset = neg_label)
_, h_idxs = k_fold_idx(n_fold = n_g, dataset = h_label)
_, d_idxs = k_fold_idx(n_fold = n_g, dataset = d_label)


h_label, h_b_output, h_e_output = tri_shuffle(h_label, h_b_output, h_e_output)
d_label, d_b_output, d_e_output = tri_shuffle(d_label, d_b_output, d_e_output)

# e_aucs, b_aucs = [], []
# e_CM, b_CM = [], []

b_metrics = []
e_metrics = []

for n_idx, h_idx, d_idx in zip(n_idxs, h_idxs, d_idxs):

    labels = np.vstack([neg_label[n_idx], h_label[h_idx], d_label[d_idx]])
    b_outputs = np.vstack([neg_b_output[n_idx], h_b_output[h_idx], d_b_output[d_idx]]) 
    e_outputs = np.vstack([neg_e_output[n_idx], h_e_output[h_idx], d_e_output[d_idx]]) 
    
    b_metrics.append(extract_5metrics(labels, b_outputs)), e_metrics.append(extract_5metrics(labels, e_outputs))
    
#     e_fpr, e_tpr, thresholds = roc_curve(labels[:, 1], e_outputs[:,1])
#     b_fpr, b_tpr, thresholds = roc_curve(labels[:, 1], b_outputs[:,1])
#     e_aucs.append(auc(e_fpr, e_tpr)), b_aucs.append(auc(b_fpr, b_tpr))

# b_metrics = []
# e_metrics = []

# for e_cm, b_cm in zip(e_CM, b_CM):
#     b_metrics.append(cls.cm2metric(b_cm))
#     e_metrics.append(cls.cm2metric(e_cm))
    
b_metrics = np.asarray(b_metrics)
e_metrics = np.asarray(e_metrics)
print('----------------------------------------')
print('Binary model VS Ensemble model')
    
print("95 % CI\n")

print('Binary model')
conf_interval(100*b_metrics[:, 0], mode = 'print', metric = 'Accr')
conf_interval(100*b_metrics[:, 1], mode = 'print', metric = 'Sen')
conf_interval(100*b_metrics[:, 2], mode = 'print', metric = 'Spec')
conf_interval(100*b_metrics[:, 3], mode = 'print', metric = 'NPV')
conf_interval(100*b_metrics[:, 4], mode = 'print', metric = 'PPV')
print('')
print('Ensemble model')
conf_interval(100*e_metrics[:, 0], mode = 'print', metric = 'Accr')
conf_interval(100*e_metrics[:, 1], mode = 'print', metric = 'Sen')
conf_interval(100*e_metrics[:, 2], mode = 'print', metric = 'Spec')
conf_interval(100*e_metrics[:, 3], mode = 'print', metric = 'NPV')
conf_interval(100*e_metrics[:, 4], mode = 'print', metric = 'PPV')    
    
# e_auc_m, e_auc_l, e_auc_u = conf_interval(e_aucs, mode = 'value')
# b_auc_m, b_auc_l, b_auc_u = conf_interval(b_aucs, mode = 'value')

# labels, b_outputs, e_outputs = tri_shuffle(labels, b_outputs, e_outputs)

# e_fpr, e_tpr, thresholds = roc_curve(labels[:, 1], e_outputs[:,1])
# b_fpr, b_tpr, thresholds = roc_curve(labels[:, 1], b_outputs[:,1])

# fig = plt.figure(figsize=(10, 8))
# plt.plot(b_fpr, b_tpr, 'k--', label="Binary model, AUC: {:.4f} ({:.4f}~{:.4f}), 95 % CI".format(b_auc_m, b_auc_l, b_auc_u))
# plt.plot(e_fpr, e_tpr, 'r--', label="Ensemble model, AUC: {:.4f} ({:.4f}~{:.4f}), 95 % CI".format(e_auc_m, e_auc_l, round(e_auc_u)))
# plt.xlabel('1 - Specificity', fontsize = 15)
# plt.ylabel('Sensitivity', fontsize = 15)
# plt.legend(fontsize = 12)
# plt.grid('on')
# fig.show()
# # fig.savefig('./results/roc_curve.png', bbox_inches='tight', dpi = 300)

# fig = plt.figure(figsize=(10, 8))
# plt.plot(b_fpr, b_tpr, 'k--', label="Binary model")
# plt.plot(e_fpr, e_tpr, 'r--', label="Ensemble model")
# plt.xlim([0.0, 0.1])
# plt.ylim([0.9, 1.0])
# plt.legend(fontsize = 20, loc = 4)
# plt.axis('off')
# fig.show()
# # fig.savefig('./results/roc_curve_enlarged.png', bbox_inches='tight', dpi = 300)

ENTIRE
------------------------------------------
Model Inference

Binary model
#: 1005 / Inference Time: 00:00:06
#: 370 / Inference Time: 00:00:02
#: 2 / Inference Time: 00:00:00
#: 293 / Inference Time: 00:00:01
#: 297 / Inference Time: 00:00:01
#: 410 / Inference Time: 00:00:02
#: 48 / Inference Time: 00:00:00

Ensemble model
#: 1005 / Inference Time: 00:00:11
#: 370 / Inference Time: 00:00:04
#: 2 / Inference Time: 00:00:00
#: 293 / Inference Time: 00:00:03
#: 297 / Inference Time: 00:00:03
#: 410 / Inference Time: 00:00:04
#: 48 / Inference Time: 00:00:00
----------------------------------------
Accucay per lesion

NH model
negative: 99.00 %
red spot: 18.38 %
angioectasia: 100.00 %
active bleeding: 41.98 %
erosion: 14.48 %
ulcer: 17.80 %
stricture: 6.25 %

ND model
negative: 99.10 %
red spot: 18.65 %
angioectasia: 50.00 %
active bleeding: 48.12 %
erosion: 19.87 %
ulcer: 22.68 %
stricture: 8.33 %

Binary model
negative: 99.20 %
red spot: 18.11 %
angioectasia: 100.00 %
active bleed

In [15]:
print('ENTIRE2')
print('------------------------------------------')
print('Model Inference')
print('')
print('Binary model')

neg_b_output = model_prob(B_2_model, neg_bgr_img)
redspot_b_output = model_prob(B_2_model, redspot_bgr_img)
angio_b_output = model_prob(B_2_model, angio_bgr_img)
active_b_output = model_prob(B_2_model, active_bgr_img)
ero_b_output = model_prob(B_2_model, ero_bgr_img)
ulcer_b_output = model_prob(B_2_model, ulcer_bgr_img)
str_b_output  = model_prob(B_2_model, str_bgr_img)

print('')
print('Ensemble model')

neg_e_output, neg_nh_output, neg_nd_output = model_prob(E_2_model, neg_bgr_img, model_type = 'ensemble')
redspot_e_output, redspot_nh_output, redspot_nd_output = model_prob(E_2_model, redspot_bgr_img, model_type = 'ensemble')
angio_e_output, angio_nh_output, angio_nd_output = model_prob(E_2_model, angio_bgr_img, model_type = 'ensemble')
active_e_output, active_nh_output, active_nd_output = model_prob(E_2_model, active_bgr_img, model_type = 'ensemble')
ero_e_output, ero_nh_output, ero_nd_output = model_prob(E_2_model, ero_bgr_img, model_type = 'ensemble')
ulcer_e_output, ulcer_nh_output, ulcer_nd_output = model_prob(E_2_model, ulcer_bgr_img, model_type = 'ensemble')
str_e_output, str_nh_output, str_nd_output  = model_prob(E_2_model, str_bgr_img, model_type = 'ensemble')

h_b_output = np.vstack([redspot_b_output, angio_b_output, active_b_output])
d_b_output = np.vstack([ero_b_output, ulcer_b_output, str_b_output])
b_outputs = np.vstack([neg_b_output, h_b_output, d_b_output])

h_e_output = np.vstack([redspot_e_output, angio_e_output, active_e_output])
d_e_output = np.vstack([ero_e_output, ulcer_e_output, str_e_output])
e_outputs = np.vstack([neg_e_output, h_e_output, d_e_output])

cls = classification_metric()
print('----------------------------------------')
print('Accucay per lesion')
print('')
print('NH model')
neg_nh_acc = cls.accuracy(neg_label, neg_nh_output, 'negative')
redspot_nh_acc = cls.accuracy(redspot_label, redspot_nh_output, 'red spot')
angio_nh_acc = cls.accuracy(angio_label, angio_nh_output, 'angioectasia')
active_nh_acc = cls.accuracy(active_label, active_nh_output, 'active bleeding')
ero_nh_acc = cls.accuracy(ero_label, ero_nh_output, 'erosion')
ulcer_nh_acc = cls.accuracy(ulcer_label, ulcer_nh_output, 'ulcer')
str_nh_acc = cls.accuracy(str_label, str_nh_output, 'stricture')

print('')
print('ND model')
neg_nd_acc = cls.accuracy(neg_label, neg_nd_output, 'negative')
redspot_nd_acc = cls.accuracy(redspot_label, redspot_nd_output, 'red spot')
angio_nd_acc = cls.accuracy(angio_label, angio_nd_output, 'angioectasia')
active_nd_acc = cls.accuracy(active_label, active_nd_output, 'active bleeding')
ero_nd_acc = cls.accuracy(ero_label, ero_nd_output, 'erosion')
ulcer_nd_acc = cls.accuracy(ulcer_label, ulcer_nd_output, 'ulcer')
str_nd_acc = cls.accuracy(str_label, str_nd_output, 'stricture')

print('')
print('Binary model')
neg_b_acc = cls.accuracy(neg_label, neg_b_output, 'negative')
redspot_b_acc = cls.accuracy(redspot_label, redspot_b_output, 'red spot')
angio_b_acc = cls.accuracy(angio_label, angio_b_output, 'angioectasia')
active_b_acc = cls.accuracy(active_label, active_b_output, 'active bleeding')
ero_b_acc = cls.accuracy(ero_label, ero_b_output, 'erosion')
ulcer_b_acc = cls.accuracy(ulcer_label, ulcer_b_output, 'ulcer')
str_b_acc = cls.accuracy(str_label, str_b_output, 'stricture')

print('')
print('Ensemble model')
neg_e_acc = cls.accuracy(neg_label, neg_e_output, 'negative')
redspot_e_acc = cls.accuracy(redspot_label, redspot_e_output, 'red spot')
angio_e_acc = cls.accuracy(angio_label, angio_e_output, 'angioectasia')
active_e_acc = cls.accuracy(active_label, active_e_output, 'active bleeding')
ero_e_acc = cls.accuracy(ero_label, ero_e_output, 'erosion')
ulcer_e_acc = cls.accuracy(ulcer_label, ulcer_e_output, 'ulcer')
str_e_acc = cls.accuracy(str_label, str_e_output, 'stricture')

n_g = 10

_, n_idxs = k_fold_idx(n_fold = n_g, dataset = neg_label)
_, h_idxs = k_fold_idx(n_fold = n_g, dataset = h_label)
_, d_idxs = k_fold_idx(n_fold = n_g, dataset = d_label)


h_label, h_b_output, h_e_output = tri_shuffle(h_label, h_b_output, h_e_output)
d_label, d_b_output, d_e_output = tri_shuffle(d_label, d_b_output, d_e_output)

# e_aucs, b_aucs = [], []
# e_CM, b_CM = [], []

b_metrics = []
e_metrics = []

for n_idx, h_idx, d_idx in zip(n_idxs, h_idxs, d_idxs):

    labels = np.vstack([neg_label[n_idx], h_label[h_idx], d_label[d_idx]])
    b_outputs = np.vstack([neg_b_output[n_idx], h_b_output[h_idx], d_b_output[d_idx]]) 
    e_outputs = np.vstack([neg_e_output[n_idx], h_e_output[h_idx], d_e_output[d_idx]]) 
    
    b_metrics.append(extract_5metrics(labels, b_outputs)), e_metrics.append(extract_5metrics(labels, e_outputs))
    
#     e_fpr, e_tpr, thresholds = roc_curve(labels[:, 1], e_outputs[:,1])
#     b_fpr, b_tpr, thresholds = roc_curve(labels[:, 1], b_outputs[:,1])
#     e_aucs.append(auc(e_fpr, e_tpr)), b_aucs.append(auc(b_fpr, b_tpr))

# b_metrics = []
# e_metrics = []

# for e_cm, b_cm in zip(e_CM, b_CM):
#     b_metrics.append(cls.cm2metric(b_cm))
#     e_metrics.append(cls.cm2metric(e_cm))
    
b_metrics = np.asarray(b_metrics)
e_metrics = np.asarray(e_metrics)
print('----------------------------------------')
print('Binary model VS Ensemble model')
    
print("95 % CI\n")

print('Binary model')
conf_interval(100*b_metrics[:, 0], mode = 'print', metric = 'Accr')
conf_interval(100*b_metrics[:, 1], mode = 'print', metric = 'Sen')
conf_interval(100*b_metrics[:, 2], mode = 'print', metric = 'Spec')
conf_interval(100*b_metrics[:, 3], mode = 'print', metric = 'NPV')
conf_interval(100*b_metrics[:, 4], mode = 'print', metric = 'PPV')
print('')
print('Ensemble model')
conf_interval(100*e_metrics[:, 0], mode = 'print', metric = 'Accr')
conf_interval(100*e_metrics[:, 1], mode = 'print', metric = 'Sen')
conf_interval(100*e_metrics[:, 2], mode = 'print', metric = 'Spec')
conf_interval(100*e_metrics[:, 3], mode = 'print', metric = 'NPV')
conf_interval(100*e_metrics[:, 4], mode = 'print', metric = 'PPV')    
    
# e_auc_m, e_auc_l, e_auc_u = conf_interval(e_aucs, mode = 'value')
# b_auc_m, b_auc_l, b_auc_u = conf_interval(b_aucs, mode = 'value')

# labels, b_outputs, e_outputs = tri_shuffle(labels, b_outputs, e_outputs)

# e_fpr, e_tpr, thresholds = roc_curve(labels[:, 1], e_outputs[:,1])
# b_fpr, b_tpr, thresholds = roc_curve(labels[:, 1], b_outputs[:,1])

# fig = plt.figure(figsize=(10, 8))
# plt.plot(b_fpr, b_tpr, 'k--', label="Binary model, AUC: {:.4f} ({:.4f}~{:.4f}), 95 % CI".format(b_auc_m, b_auc_l, b_auc_u))
# plt.plot(e_fpr, e_tpr, 'r--', label="Ensemble model, AUC: {:.4f} ({:.4f}~{:.4f}), 95 % CI".format(e_auc_m, e_auc_l, round(e_auc_u)))
# plt.xlabel('1 - Specificity', fontsize = 15)
# plt.ylabel('Sensitivity', fontsize = 15)
# plt.legend(fontsize = 12)
# plt.grid('on')
# fig.show()
# # fig.savefig('./results/roc_curve.png', bbox_inches='tight', dpi = 300)

# fig = plt.figure(figsize=(10, 8))
# plt.plot(b_fpr, b_tpr, 'k--', label="Binary model")
# plt.plot(e_fpr, e_tpr, 'r--', label="Ensemble model")
# plt.xlim([0.0, 0.1])
# plt.ylim([0.9, 1.0])
# plt.legend(fontsize = 20, loc = 4)
# plt.axis('off')
# fig.show()
# # fig.savefig('./results/roc_curve_enlarged.png', bbox_inches='tight', dpi = 300)

ENTIRE2
------------------------------------------
Model Inference

Binary model
#: 1005 / Inference Time: 00:00:05
#: 370 / Inference Time: 00:00:02
#: 2 / Inference Time: 00:00:00
#: 293 / Inference Time: 00:00:01
#: 297 / Inference Time: 00:00:01
#: 410 / Inference Time: 00:00:02
#: 48 / Inference Time: 00:00:00

Ensemble model
#: 1005 / Inference Time: 00:00:10
#: 370 / Inference Time: 00:00:04
#: 2 / Inference Time: 00:00:00
#: 293 / Inference Time: 00:00:03
#: 297 / Inference Time: 00:00:03
#: 410 / Inference Time: 00:00:04
#: 48 / Inference Time: 00:00:00
----------------------------------------
Accucay per lesion

NH model
negative: 94.43 %
red spot: 79.46 %
angioectasia: 100.00 %
active bleeding: 92.83 %
erosion: 51.52 %
ulcer: 64.63 %
stricture: 41.67 %

ND model
negative: 88.76 %
red spot: 68.11 %
angioectasia: 100.00 %
active bleeding: 87.03 %
erosion: 79.12 %
ulcer: 80.00 %
stricture: 75.00 %

Binary model
negative: 88.36 %
red spot: 90.54 %
angioectasia: 100.00 %
active b

In [16]:
print('ENTIRE(TOTAL)')
print('------------------------------------------')
print('Model Inference')
print('')
print('Binary model')

neg_b_output = model_prob(B_t_model, neg_bgr_img)
redspot_b_output = model_prob(B_t_model, redspot_bgr_img)
angio_b_output = model_prob(B_t_model, angio_bgr_img)
active_b_output = model_prob(B_t_model, active_bgr_img)
ero_b_output = model_prob(B_t_model, ero_bgr_img)
ulcer_b_output = model_prob(B_t_model, ulcer_bgr_img)
str_b_output  = model_prob(B_t_model, str_bgr_img)

print('')
print('Ensemble model')

neg_e_output, neg_nh_output, neg_nd_output = model_prob(E_t_model, neg_bgr_img, model_type = 'ensemble')
redspot_e_output, redspot_nh_output, redspot_nd_output = model_prob(E_t_model, redspot_bgr_img, model_type = 'ensemble')
angio_e_output, angio_nh_output, angio_nd_output = model_prob(E_t_model, angio_bgr_img, model_type = 'ensemble')
active_e_output, active_nh_output, active_nd_output = model_prob(E_t_model, active_bgr_img, model_type = 'ensemble')
ero_e_output, ero_nh_output, ero_nd_output = model_prob(E_t_model, ero_bgr_img, model_type = 'ensemble')
ulcer_e_output, ulcer_nh_output, ulcer_nd_output = model_prob(E_t_model, ulcer_bgr_img, model_type = 'ensemble')
str_e_output, str_nh_output, str_nd_output  = model_prob(E_t_model, str_bgr_img, model_type = 'ensemble')

h_b_output = np.vstack([redspot_b_output, angio_b_output, active_b_output])
d_b_output = np.vstack([ero_b_output, ulcer_b_output, str_b_output])
b_outputs = np.vstack([neg_b_output, h_b_output, d_b_output])

h_e_output = np.vstack([redspot_e_output, angio_e_output, active_e_output])
d_e_output = np.vstack([ero_e_output, ulcer_e_output, str_e_output])
e_outputs = np.vstack([neg_e_output, h_e_output, d_e_output])

cls = classification_metric()
print('----------------------------------------')
print('Accucay per lesion')
print('')
print('NH model')
neg_nh_acc = cls.accuracy(neg_label, neg_nh_output, 'negative')
redspot_nh_acc = cls.accuracy(redspot_label, redspot_nh_output, 'red spot')
angio_nh_acc = cls.accuracy(angio_label, angio_nh_output, 'angioectasia')
active_nh_acc = cls.accuracy(active_label, active_nh_output, 'active bleeding')
ero_nh_acc = cls.accuracy(ero_label, ero_nh_output, 'erosion')
ulcer_nh_acc = cls.accuracy(ulcer_label, ulcer_nh_output, 'ulcer')
str_nh_acc = cls.accuracy(str_label, str_nh_output, 'stricture')

print('')
print('ND model')
neg_nd_acc = cls.accuracy(neg_label, neg_nd_output, 'negative')
redspot_nd_acc = cls.accuracy(redspot_label, redspot_nd_output, 'red spot')
angio_nd_acc = cls.accuracy(angio_label, angio_nd_output, 'angioectasia')
active_nd_acc = cls.accuracy(active_label, active_nd_output, 'active bleeding')
ero_nd_acc = cls.accuracy(ero_label, ero_nd_output, 'erosion')
ulcer_nd_acc = cls.accuracy(ulcer_label, ulcer_nd_output, 'ulcer')
str_nd_acc = cls.accuracy(str_label, str_nd_output, 'stricture')

print('')
print('Binary model')
neg_b_acc = cls.accuracy(neg_label, neg_b_output, 'negative')
redspot_b_acc = cls.accuracy(redspot_label, redspot_b_output, 'red spot')
angio_b_acc = cls.accuracy(angio_label, angio_b_output, 'angioectasia')
active_b_acc = cls.accuracy(active_label, active_b_output, 'active bleeding')
ero_b_acc = cls.accuracy(ero_label, ero_b_output, 'erosion')
ulcer_b_acc = cls.accuracy(ulcer_label, ulcer_b_output, 'ulcer')
str_b_acc = cls.accuracy(str_label, str_b_output, 'stricture')

print('')
print('Ensemble model')
neg_e_acc = cls.accuracy(neg_label, neg_e_output, 'negative')
redspot_e_acc = cls.accuracy(redspot_label, redspot_e_output, 'red spot')
angio_e_acc = cls.accuracy(angio_label, angio_e_output, 'angioectasia')
active_e_acc = cls.accuracy(active_label, active_e_output, 'active bleeding')
ero_e_acc = cls.accuracy(ero_label, ero_e_output, 'erosion')
ulcer_e_acc = cls.accuracy(ulcer_label, ulcer_e_output, 'ulcer')
str_e_acc = cls.accuracy(str_label, str_e_output, 'stricture')

n_g = 10

_, n_idxs = k_fold_idx(n_fold = n_g, dataset = neg_label)
_, h_idxs = k_fold_idx(n_fold = n_g, dataset = h_label)
_, d_idxs = k_fold_idx(n_fold = n_g, dataset = d_label)


h_label, h_b_output, h_e_output = tri_shuffle(h_label, h_b_output, h_e_output)
d_label, d_b_output, d_e_output = tri_shuffle(d_label, d_b_output, d_e_output)

# e_aucs, b_aucs = [], []
# e_CM, b_CM = [], []

b_metrics = []
e_metrics = []

for n_idx, h_idx, d_idx in zip(n_idxs, h_idxs, d_idxs):

    labels = np.vstack([neg_label[n_idx], h_label[h_idx], d_label[d_idx]])
    b_outputs = np.vstack([neg_b_output[n_idx], h_b_output[h_idx], d_b_output[d_idx]]) 
    e_outputs = np.vstack([neg_e_output[n_idx], h_e_output[h_idx], d_e_output[d_idx]]) 
    
    b_metrics.append(extract_5metrics(labels, b_outputs)), e_metrics.append(extract_5metrics(labels, e_outputs))
    
#     e_fpr, e_tpr, thresholds = roc_curve(labels[:, 1], e_outputs[:,1])
#     b_fpr, b_tpr, thresholds = roc_curve(labels[:, 1], b_outputs[:,1])
#     e_aucs.append(auc(e_fpr, e_tpr)), b_aucs.append(auc(b_fpr, b_tpr))

# b_metrics = []
# e_metrics = []

# for e_cm, b_cm in zip(e_CM, b_CM):
#     b_metrics.append(cls.cm2metric(b_cm))
#     e_metrics.append(cls.cm2metric(e_cm))
    
b_metrics = np.asarray(b_metrics)
e_metrics = np.asarray(e_metrics)
print('----------------------------------------')
print('Binary model VS Ensemble model')
    
print("95 % CI\n")

print('Binary model')
conf_interval(100*b_metrics[:, 0], mode = 'print', metric = 'Accr')
conf_interval(100*b_metrics[:, 1], mode = 'print', metric = 'Sen')
conf_interval(100*b_metrics[:, 2], mode = 'print', metric = 'Spec')
conf_interval(100*b_metrics[:, 3], mode = 'print', metric = 'NPV')
conf_interval(100*b_metrics[:, 4], mode = 'print', metric = 'PPV')
print('')
print('Ensemble model')
conf_interval(100*e_metrics[:, 0], mode = 'print', metric = 'Accr')
conf_interval(100*e_metrics[:, 1], mode = 'print', metric = 'Sen')
conf_interval(100*e_metrics[:, 2], mode = 'print', metric = 'Spec')
conf_interval(100*e_metrics[:, 3], mode = 'print', metric = 'NPV')
conf_interval(100*e_metrics[:, 4], mode = 'print', metric = 'PPV')    
    
# e_auc_m, e_auc_l, e_auc_u = conf_interval(e_aucs, mode = 'value')
# b_auc_m, b_auc_l, b_auc_u = conf_interval(b_aucs, mode = 'value')

# labels, b_outputs, e_outputs = tri_shuffle(labels, b_outputs, e_outputs)

# e_fpr, e_tpr, thresholds = roc_curve(labels[:, 1], e_outputs[:,1])
# b_fpr, b_tpr, thresholds = roc_curve(labels[:, 1], b_outputs[:,1])

# fig = plt.figure(figsize=(10, 8))
# plt.plot(b_fpr, b_tpr, 'k--', label="Binary model, AUC: {:.4f} ({:.4f}~{:.4f}), 95 % CI".format(b_auc_m, b_auc_l, b_auc_u))
# plt.plot(e_fpr, e_tpr, 'r--', label="Ensemble model, AUC: {:.4f} ({:.4f}~{:.4f}), 95 % CI".format(e_auc_m, e_auc_l, round(e_auc_u)))
# plt.xlabel('1 - Specificity', fontsize = 15)
# plt.ylabel('Sensitivity', fontsize = 15)
# plt.legend(fontsize = 12)
# plt.grid('on')
# fig.show()
# # fig.savefig('./results/roc_curve.png', bbox_inches='tight', dpi = 300)

# fig = plt.figure(figsize=(10, 8))
# plt.plot(b_fpr, b_tpr, 'k--', label="Binary model")
# plt.plot(e_fpr, e_tpr, 'r--', label="Ensemble model")
# plt.xlim([0.0, 0.1])
# plt.ylim([0.9, 1.0])
# plt.legend(fontsize = 20, loc = 4)
# plt.axis('off')
# fig.show()
# # fig.savefig('./results/roc_curve_enlarged.png', bbox_inches='tight', dpi = 300)

ENTIRE(TOTAL)
------------------------------------------
Model Inference

Binary model
#: 1005 / Inference Time: 00:00:05
#: 370 / Inference Time: 00:00:02
#: 2 / Inference Time: 00:00:00
#: 293 / Inference Time: 00:00:01
#: 297 / Inference Time: 00:00:01
#: 410 / Inference Time: 00:00:02
#: 48 / Inference Time: 00:00:00

Ensemble model
#: 1005 / Inference Time: 00:00:11
#: 370 / Inference Time: 00:00:04
#: 2 / Inference Time: 00:00:00
#: 293 / Inference Time: 00:00:03
#: 297 / Inference Time: 00:00:03
#: 410 / Inference Time: 00:00:04
#: 48 / Inference Time: 00:00:00
----------------------------------------
Accucay per lesion

NH model
negative: 99.40 %
red spot: 28.11 %
angioectasia: 100.00 %
active bleeding: 77.82 %
erosion: 19.87 %
ulcer: 22.93 %
stricture: 14.58 %

ND model
negative: 99.20 %
red spot: 9.46 %
angioectasia: 50.00 %
active bleeding: 35.84 %
erosion: 11.11 %
ulcer: 12.44 %
stricture: 12.50 %

Binary model
negative: 98.31 %
red spot: 27.57 %
angioectasia: 100.00 %
acti

In [17]:
nh_x160_list = sorted([i[:-5] for i in os.listdir('./model/X160_v2') if 'meta' in i and 'X160_v2_fr_NH' in i])
nd_x160_list = sorted([i[:-5] for i in os.listdir('./model/X160_v2') if 'meta' in i and 'X160_v2_fr_ND' in i])
b_x160_list = sorted([i[:-5] for i in os.listdir('./model/X160_v2') if 'meta' in i and 'X160_v2_fr_B' in i])

nh_path = './model/X160_v2/' + nh_x160_list[-1]
nd_path = './model/X160_v2/' + nd_x160_list[-1]
b_path = './model/X160_v2/' + b_x160_list[-1]

NH_x160_model = binary_model(nh_path)
ND_x160_model = binary_model(nd_path)
E_x160_model = ensemble_model(nh_path, nd_path)
B_x160_model = binary_model(b_path)

In [18]:
print('X160_v2 fr')
print('------------------------------------------')
print('Model Inference')
print('')
print('Binary model')

neg_b_output = model_prob(B_x160_model, neg_bgr_img)
redspot_b_output = model_prob(B_x160_model, redspot_bgr_img)
angio_b_output = model_prob(B_x160_model, angio_bgr_img)
active_b_output = model_prob(B_x160_model, active_bgr_img)
ero_b_output = model_prob(B_x160_model, ero_bgr_img)
ulcer_b_output = model_prob(B_x160_model, ulcer_bgr_img)
str_b_output  = model_prob(B_x160_model, str_bgr_img)

print('')
print('Ensemble model')

neg_e_output, neg_nh_output, neg_nd_output = model_prob(E_x160_model, neg_bgr_img, model_type = 'ensemble')
redspot_e_output, redspot_nh_output, redspot_nd_output = model_prob(E_x160_model, redspot_bgr_img, model_type = 'ensemble')
angio_e_output, angio_nh_output, angio_nd_output = model_prob(E_x160_model, angio_bgr_img, model_type = 'ensemble')
active_e_output, active_nh_output, active_nd_output = model_prob(E_x160_model, active_bgr_img, model_type = 'ensemble')
ero_e_output, ero_nh_output, ero_nd_output = model_prob(E_x160_model, ero_bgr_img, model_type = 'ensemble')
ulcer_e_output, ulcer_nh_output, ulcer_nd_output = model_prob(E_x160_model, ulcer_bgr_img, model_type = 'ensemble')
str_e_output, str_nh_output, str_nd_output  = model_prob(E_x160_model, str_bgr_img, model_type = 'ensemble')

h_b_output = np.vstack([redspot_b_output, angio_b_output, active_b_output])
d_b_output = np.vstack([ero_b_output, ulcer_b_output, str_b_output])
b_outputs = np.vstack([neg_b_output, h_b_output, d_b_output])

h_e_output = np.vstack([redspot_e_output, angio_e_output, active_e_output])
d_e_output = np.vstack([ero_e_output, ulcer_e_output, str_e_output])
e_outputs = np.vstack([neg_e_output, h_e_output, d_e_output])

cls = classification_metric()
print('----------------------------------------')
print('Accucay per lesion')
print('')
print('NH model')
neg_nh_acc = cls.accuracy(neg_label, neg_nh_output, 'negative')
redspot_nh_acc = cls.accuracy(redspot_label, redspot_nh_output, 'red spot')
angio_nh_acc = cls.accuracy(angio_label, angio_nh_output, 'angioectasia')
active_nh_acc = cls.accuracy(active_label, active_nh_output, 'active bleeding')
ero_nh_acc = cls.accuracy(ero_label, ero_nh_output, 'erosion')
ulcer_nh_acc = cls.accuracy(ulcer_label, ulcer_nh_output, 'ulcer')
str_nh_acc = cls.accuracy(str_label, str_nh_output, 'stricture')

print('')
print('ND model')
neg_nd_acc = cls.accuracy(neg_label, neg_nd_output, 'negative')
redspot_nd_acc = cls.accuracy(redspot_label, redspot_nd_output, 'red spot')
angio_nd_acc = cls.accuracy(angio_label, angio_nd_output, 'angioectasia')
active_nd_acc = cls.accuracy(active_label, active_nd_output, 'active bleeding')
ero_nd_acc = cls.accuracy(ero_label, ero_nd_output, 'erosion')
ulcer_nd_acc = cls.accuracy(ulcer_label, ulcer_nd_output, 'ulcer')
str_nd_acc = cls.accuracy(str_label, str_nd_output, 'stricture')

print('')
print('Binary model')
neg_b_acc = cls.accuracy(neg_label, neg_b_output, 'negative')
redspot_b_acc = cls.accuracy(redspot_label, redspot_b_output, 'red spot')
angio_b_acc = cls.accuracy(angio_label, angio_b_output, 'angioectasia')
active_b_acc = cls.accuracy(active_label, active_b_output, 'active bleeding')
ero_b_acc = cls.accuracy(ero_label, ero_b_output, 'erosion')
ulcer_b_acc = cls.accuracy(ulcer_label, ulcer_b_output, 'ulcer')
str_b_acc = cls.accuracy(str_label, str_b_output, 'stricture')

print('')
print('Ensemble model')
neg_e_acc = cls.accuracy(neg_label, neg_e_output, 'negative')
redspot_e_acc = cls.accuracy(redspot_label, redspot_e_output, 'red spot')
angio_e_acc = cls.accuracy(angio_label, angio_e_output, 'angioectasia')
active_e_acc = cls.accuracy(active_label, active_e_output, 'active bleeding')
ero_e_acc = cls.accuracy(ero_label, ero_e_output, 'erosion')
ulcer_e_acc = cls.accuracy(ulcer_label, ulcer_e_output, 'ulcer')
str_e_acc = cls.accuracy(str_label, str_e_output, 'stricture')

n_g = 10

_, n_idxs = k_fold_idx(n_fold = n_g, dataset = neg_label)
_, h_idxs = k_fold_idx(n_fold = n_g, dataset = h_label)
_, d_idxs = k_fold_idx(n_fold = n_g, dataset = d_label)


h_label, h_b_output, h_e_output = tri_shuffle(h_label, h_b_output, h_e_output)
d_label, d_b_output, d_e_output = tri_shuffle(d_label, d_b_output, d_e_output)

# e_aucs, b_aucs = [], []
# e_CM, b_CM = [], []

b_metrics = []
e_metrics = []

for n_idx, h_idx, d_idx in zip(n_idxs, h_idxs, d_idxs):

    labels = np.vstack([neg_label[n_idx], h_label[h_idx], d_label[d_idx]])
    b_outputs = np.vstack([neg_b_output[n_idx], h_b_output[h_idx], d_b_output[d_idx]]) 
    e_outputs = np.vstack([neg_e_output[n_idx], h_e_output[h_idx], d_e_output[d_idx]]) 
    
    b_metrics.append(extract_5metrics(labels, b_outputs)), e_metrics.append(extract_5metrics(labels, e_outputs))
    
#     e_fpr, e_tpr, thresholds = roc_curve(labels[:, 1], e_outputs[:,1])
#     b_fpr, b_tpr, thresholds = roc_curve(labels[:, 1], b_outputs[:,1])
#     e_aucs.append(auc(e_fpr, e_tpr)), b_aucs.append(auc(b_fpr, b_tpr))

# b_metrics = []
# e_metrics = []

# for e_cm, b_cm in zip(e_CM, b_CM):
#     b_metrics.append(cls.cm2metric(b_cm))
#     e_metrics.append(cls.cm2metric(e_cm))
    
b_metrics = np.asarray(b_metrics)
e_metrics = np.asarray(e_metrics)
print('----------------------------------------')
print('Binary model VS Ensemble model')
    
print("95 % CI\n")

print('Binary model')
conf_interval(100*b_metrics[:, 0], mode = 'print', metric = 'Accr')
conf_interval(100*b_metrics[:, 1], mode = 'print', metric = 'Sen')
conf_interval(100*b_metrics[:, 2], mode = 'print', metric = 'Spec')
conf_interval(100*b_metrics[:, 3], mode = 'print', metric = 'NPV')
conf_interval(100*b_metrics[:, 4], mode = 'print', metric = 'PPV')
print('')
print('Ensemble model')
conf_interval(100*e_metrics[:, 0], mode = 'print', metric = 'Accr')
conf_interval(100*e_metrics[:, 1], mode = 'print', metric = 'Sen')
conf_interval(100*e_metrics[:, 2], mode = 'print', metric = 'Spec')
conf_interval(100*e_metrics[:, 3], mode = 'print', metric = 'NPV')
conf_interval(100*e_metrics[:, 4], mode = 'print', metric = 'PPV')    
    
# e_auc_m, e_auc_l, e_auc_u = conf_interval(e_aucs, mode = 'value')
# b_auc_m, b_auc_l, b_auc_u = conf_interval(b_aucs, mode = 'value')

# labels, b_outputs, e_outputs = tri_shuffle(labels, b_outputs, e_outputs)

# e_fpr, e_tpr, thresholds = roc_curve(labels[:, 1], e_outputs[:,1])
# b_fpr, b_tpr, thresholds = roc_curve(labels[:, 1], b_outputs[:,1])

# fig = plt.figure(figsize=(10, 8))
# plt.plot(b_fpr, b_tpr, 'k--', label="Binary model, AUC: {:.4f} ({:.4f}~{:.4f}), 95 % CI".format(b_auc_m, b_auc_l, b_auc_u))
# plt.plot(e_fpr, e_tpr, 'r--', label="Ensemble model, AUC: {:.4f} ({:.4f}~{:.4f}), 95 % CI".format(e_auc_m, e_auc_l, round(e_auc_u)))
# plt.xlabel('1 - Specificity', fontsize = 15)
# plt.ylabel('Sensitivity', fontsize = 15)
# plt.legend(fontsize = 12)
# plt.grid('on')
# fig.show()
# # fig.savefig('./results/roc_curve.png', bbox_inches='tight', dpi = 300)

# fig = plt.figure(figsize=(10, 8))
# plt.plot(b_fpr, b_tpr, 'k--', label="Binary model")
# plt.plot(e_fpr, e_tpr, 'r--', label="Ensemble model")
# plt.xlim([0.0, 0.1])
# plt.ylim([0.9, 1.0])
# plt.legend(fontsize = 20, loc = 4)
# plt.axis('off')
# fig.show()
# # fig.savefig('./results/roc_curve_enlarged.png', bbox_inches='tight', dpi = 300)

X160_v2 fr
------------------------------------------
Model Inference

Binary model
#: 1005 / Inference Time: 00:00:06
#: 370 / Inference Time: 00:00:02
#: 2 / Inference Time: 00:00:00
#: 293 / Inference Time: 00:00:01
#: 297 / Inference Time: 00:00:01
#: 410 / Inference Time: 00:00:02
#: 48 / Inference Time: 00:00:00

Ensemble model
#: 1005 / Inference Time: 00:00:12
#: 370 / Inference Time: 00:00:04
#: 2 / Inference Time: 00:00:00
#: 293 / Inference Time: 00:00:03
#: 297 / Inference Time: 00:00:03
#: 410 / Inference Time: 00:00:04
#: 48 / Inference Time: 00:00:00
----------------------------------------
Accucay per lesion

NH model
negative: 87.26 %
red spot: 88.92 %
angioectasia: 100.00 %
active bleeding: 95.56 %
erosion: 58.25 %
ulcer: 76.10 %
stricture: 60.42 %

ND model
negative: 82.29 %
red spot: 65.95 %
angioectasia: 100.00 %
active bleeding: 82.94 %
erosion: 74.07 %
ulcer: 79.27 %
stricture: 66.67 %

Binary model
negative: 57.61 %
red spot: 92.97 %
angioectasia: 100.00 %
activ

In [19]:
nh_x160_list = sorted([i[:-5] for i in os.listdir('./model/X160_v2') if 'meta' in i and 'X160_v2_frbsv_NH' in i])
nd_x160_list = sorted([i[:-5] for i in os.listdir('./model/X160_v2') if 'meta' in i and 'X160_v2_frbsv_ND' in i])
b_x160_list = sorted([i[:-5] for i in os.listdir('./model/X160_v2') if 'meta' in i and 'X160_v2_frbsv_B' in i])

nh_path = './model/X160_v2/' + nh_x160_list[-1]
nd_path = './model/X160_v2/' + nd_x160_list[-1]
b_path = './model/X160_v2/' + b_x160_list[-1]

NH_x160_model = binary_model(nh_path)
ND_x160_model = binary_model(nd_path)
E_x160_model = ensemble_model(nh_path, nd_path)
B_x160_model = binary_model(b_path)

In [20]:
print('X160_v2 frbsv')
print('------------------------------------------')
print('Model Inference')
print('')
print('Binary model')

neg_b_output = model_prob(B_x160_model, neg_bgr_img)
redspot_b_output = model_prob(B_x160_model, redspot_bgr_img)
angio_b_output = model_prob(B_x160_model, angio_bgr_img)
active_b_output = model_prob(B_x160_model, active_bgr_img)
ero_b_output = model_prob(B_x160_model, ero_bgr_img)
ulcer_b_output = model_prob(B_x160_model, ulcer_bgr_img)
str_b_output  = model_prob(B_x160_model, str_bgr_img)

print('')
print('Ensemble model')

neg_e_output, neg_nh_output, neg_nd_output = model_prob(E_x160_model, neg_bgr_img, model_type = 'ensemble')
redspot_e_output, redspot_nh_output, redspot_nd_output = model_prob(E_x160_model, redspot_bgr_img, model_type = 'ensemble')
angio_e_output, angio_nh_output, angio_nd_output = model_prob(E_x160_model, angio_bgr_img, model_type = 'ensemble')
active_e_output, active_nh_output, active_nd_output = model_prob(E_x160_model, active_bgr_img, model_type = 'ensemble')
ero_e_output, ero_nh_output, ero_nd_output = model_prob(E_x160_model, ero_bgr_img, model_type = 'ensemble')
ulcer_e_output, ulcer_nh_output, ulcer_nd_output = model_prob(E_x160_model, ulcer_bgr_img, model_type = 'ensemble')
str_e_output, str_nh_output, str_nd_output  = model_prob(E_x160_model, str_bgr_img, model_type = 'ensemble')

h_b_output = np.vstack([redspot_b_output, angio_b_output, active_b_output])
d_b_output = np.vstack([ero_b_output, ulcer_b_output, str_b_output])
b_outputs = np.vstack([neg_b_output, h_b_output, d_b_output])

h_e_output = np.vstack([redspot_e_output, angio_e_output, active_e_output])
d_e_output = np.vstack([ero_e_output, ulcer_e_output, str_e_output])
e_outputs = np.vstack([neg_e_output, h_e_output, d_e_output])

cls = classification_metric()
print('----------------------------------------')
print('Accucay per lesion')
print('')
print('NH model')
neg_nh_acc = cls.accuracy(neg_label, neg_nh_output, 'negative')
redspot_nh_acc = cls.accuracy(redspot_label, redspot_nh_output, 'red spot')
angio_nh_acc = cls.accuracy(angio_label, angio_nh_output, 'angioectasia')
active_nh_acc = cls.accuracy(active_label, active_nh_output, 'active bleeding')
ero_nh_acc = cls.accuracy(ero_label, ero_nh_output, 'erosion')
ulcer_nh_acc = cls.accuracy(ulcer_label, ulcer_nh_output, 'ulcer')
str_nh_acc = cls.accuracy(str_label, str_nh_output, 'stricture')

print('')
print('ND model')
neg_nd_acc = cls.accuracy(neg_label, neg_nd_output, 'negative')
redspot_nd_acc = cls.accuracy(redspot_label, redspot_nd_output, 'red spot')
angio_nd_acc = cls.accuracy(angio_label, angio_nd_output, 'angioectasia')
active_nd_acc = cls.accuracy(active_label, active_nd_output, 'active bleeding')
ero_nd_acc = cls.accuracy(ero_label, ero_nd_output, 'erosion')
ulcer_nd_acc = cls.accuracy(ulcer_label, ulcer_nd_output, 'ulcer')
str_nd_acc = cls.accuracy(str_label, str_nd_output, 'stricture')

print('')
print('Binary model')
neg_b_acc = cls.accuracy(neg_label, neg_b_output, 'negative')
redspot_b_acc = cls.accuracy(redspot_label, redspot_b_output, 'red spot')
angio_b_acc = cls.accuracy(angio_label, angio_b_output, 'angioectasia')
active_b_acc = cls.accuracy(active_label, active_b_output, 'active bleeding')
ero_b_acc = cls.accuracy(ero_label, ero_b_output, 'erosion')
ulcer_b_acc = cls.accuracy(ulcer_label, ulcer_b_output, 'ulcer')
str_b_acc = cls.accuracy(str_label, str_b_output, 'stricture')

print('')
print('Ensemble model')
neg_e_acc = cls.accuracy(neg_label, neg_e_output, 'negative')
redspot_e_acc = cls.accuracy(redspot_label, redspot_e_output, 'red spot')
angio_e_acc = cls.accuracy(angio_label, angio_e_output, 'angioectasia')
active_e_acc = cls.accuracy(active_label, active_e_output, 'active bleeding')
ero_e_acc = cls.accuracy(ero_label, ero_e_output, 'erosion')
ulcer_e_acc = cls.accuracy(ulcer_label, ulcer_e_output, 'ulcer')
str_e_acc = cls.accuracy(str_label, str_e_output, 'stricture')

n_g = 10

_, n_idxs = k_fold_idx(n_fold = n_g, dataset = neg_label)
_, h_idxs = k_fold_idx(n_fold = n_g, dataset = h_label)
_, d_idxs = k_fold_idx(n_fold = n_g, dataset = d_label)


h_label, h_b_output, h_e_output = tri_shuffle(h_label, h_b_output, h_e_output)
d_label, d_b_output, d_e_output = tri_shuffle(d_label, d_b_output, d_e_output)

# e_aucs, b_aucs = [], []
# e_CM, b_CM = [], []

b_metrics = []
e_metrics = []

for n_idx, h_idx, d_idx in zip(n_idxs, h_idxs, d_idxs):

    labels = np.vstack([neg_label[n_idx], h_label[h_idx], d_label[d_idx]])
    b_outputs = np.vstack([neg_b_output[n_idx], h_b_output[h_idx], d_b_output[d_idx]]) 
    e_outputs = np.vstack([neg_e_output[n_idx], h_e_output[h_idx], d_e_output[d_idx]]) 
    
    b_metrics.append(extract_5metrics(labels, b_outputs)), e_metrics.append(extract_5metrics(labels, e_outputs))
    
#     e_fpr, e_tpr, thresholds = roc_curve(labels[:, 1], e_outputs[:,1])
#     b_fpr, b_tpr, thresholds = roc_curve(labels[:, 1], b_outputs[:,1])
#     e_aucs.append(auc(e_fpr, e_tpr)), b_aucs.append(auc(b_fpr, b_tpr))

# b_metrics = []
# e_metrics = []

# for e_cm, b_cm in zip(e_CM, b_CM):
#     b_metrics.append(cls.cm2metric(b_cm))
#     e_metrics.append(cls.cm2metric(e_cm))
    
b_metrics = np.asarray(b_metrics)
e_metrics = np.asarray(e_metrics)
print('----------------------------------------')
print('Binary model VS Ensemble model')
    
print("95 % CI\n")

print('Binary model')
conf_interval(100*b_metrics[:, 0], mode = 'print', metric = 'Accr')
conf_interval(100*b_metrics[:, 1], mode = 'print', metric = 'Sen')
conf_interval(100*b_metrics[:, 2], mode = 'print', metric = 'Spec')
conf_interval(100*b_metrics[:, 3], mode = 'print', metric = 'NPV')
conf_interval(100*b_metrics[:, 4], mode = 'print', metric = 'PPV')
print('')
print('Ensemble model')
conf_interval(100*e_metrics[:, 0], mode = 'print', metric = 'Accr')
conf_interval(100*e_metrics[:, 1], mode = 'print', metric = 'Sen')
conf_interval(100*e_metrics[:, 2], mode = 'print', metric = 'Spec')
conf_interval(100*e_metrics[:, 3], mode = 'print', metric = 'NPV')
conf_interval(100*e_metrics[:, 4], mode = 'print', metric = 'PPV')    
    
# e_auc_m, e_auc_l, e_auc_u = conf_interval(e_aucs, mode = 'value')
# b_auc_m, b_auc_l, b_auc_u = conf_interval(b_aucs, mode = 'value')

# labels, b_outputs, e_outputs = tri_shuffle(labels, b_outputs, e_outputs)

# e_fpr, e_tpr, thresholds = roc_curve(labels[:, 1], e_outputs[:,1])
# b_fpr, b_tpr, thresholds = roc_curve(labels[:, 1], b_outputs[:,1])

# fig = plt.figure(figsize=(10, 8))
# plt.plot(b_fpr, b_tpr, 'k--', label="Binary model, AUC: {:.4f} ({:.4f}~{:.4f}), 95 % CI".format(b_auc_m, b_auc_l, b_auc_u))
# plt.plot(e_fpr, e_tpr, 'r--', label="Ensemble model, AUC: {:.4f} ({:.4f}~{:.4f}), 95 % CI".format(e_auc_m, e_auc_l, round(e_auc_u)))
# plt.xlabel('1 - Specificity', fontsize = 15)
# plt.ylabel('Sensitivity', fontsize = 15)
# plt.legend(fontsize = 12)
# plt.grid('on')
# fig.show()
# # fig.savefig('./results/roc_curve.png', bbox_inches='tight', dpi = 300)

# fig = plt.figure(figsize=(10, 8))
# plt.plot(b_fpr, b_tpr, 'k--', label="Binary model")
# plt.plot(e_fpr, e_tpr, 'r--', label="Ensemble model")
# plt.xlim([0.0, 0.1])
# plt.ylim([0.9, 1.0])
# plt.legend(fontsize = 20, loc = 4)
# plt.axis('off')
# fig.show()
# # fig.savefig('./results/roc_curve_enlarged.png', bbox_inches='tight', dpi = 300)

X160_v2 frbsv
------------------------------------------
Model Inference

Binary model
#: 1005 / Inference Time: 00:00:05
#: 370 / Inference Time: 00:00:02
#: 2 / Inference Time: 00:00:00
#: 293 / Inference Time: 00:00:01
#: 297 / Inference Time: 00:00:01
#: 410 / Inference Time: 00:00:02
#: 48 / Inference Time: 00:00:00

Ensemble model
#: 1005 / Inference Time: 00:00:11
#: 370 / Inference Time: 00:00:04
#: 2 / Inference Time: 00:00:00
#: 293 / Inference Time: 00:00:03
#: 297 / Inference Time: 00:00:03
#: 410 / Inference Time: 00:00:04
#: 48 / Inference Time: 00:00:00
----------------------------------------
Accucay per lesion

NH model
negative: 85.07 %
red spot: 88.38 %
angioectasia: 100.00 %
active bleeding: 96.25 %
erosion: 62.96 %
ulcer: 76.83 %
stricture: 56.25 %

ND model
negative: 83.58 %
red spot: 72.43 %
angioectasia: 100.00 %
active bleeding: 88.05 %
erosion: 80.81 %
ulcer: 80.24 %
stricture: 66.67 %

Binary model
negative: 77.91 %
red spot: 91.08 %
angioectasia: 100.00 %
ac